In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from google.cloud import bigquery
import datetime as dt

In [4]:
def get_df_from_bq(project_name, query):
    """
    configurar primeiro a CLI do google cloud 
    """
    client = bigquery.Client(project=project_name)
    query_job = client.query(query)
    df = query_job.result().to_dataframe()

    return df

In [5]:
query = "SELECT * FROM `windproject-393609.FAOSTAT.crop_yield`"
project_name = "windproject-393609"
df = get_df_from_bq(project_name, query)

In [7]:
df.head()

,Domain_Code,Domain,Area_Code__M49_,Area,Element_Code,Element,Item_Code__CPC_,Item,Year_Code,Year,Unit,Value,Flag,Flag_Description
0,QCL,Crops and livestock products,4,Afghanistan,5419,Yield,1312,Bananas,1961,1961,,NaN,,
1,QCL,Crops and livestock products,4,Afghanistan,5419,Yield,1312,Bananas,1962,1962,,NaN,,
2,QCL,Crops and livestock products,4,Afghanistan,5419,Yield,1312,Bananas,1963,1963,,NaN,,
3,QCL,Crops and livestock products,4,Afghanistan,5419,Yield,1312,Bananas,1964,1964,,NaN,,
4,QCL,Crops and livestock products,4,Afghanistan,5419,Yield,1312,Bananas,1965,1965,,NaN,,


In [9]:
df.drop(columns = ["Domain_Code","Area_Code__M49_","Element_Code","Item_Code__CPC_","Year_Code"], inplace = True)

In [10]:
df.shape

(115839, 9)

In [11]:
df.Domain.value_counts()

Domain
Crops and livestock products    115839
Name: count, dtype: int64

In [12]:
df.Area.value_counts()

Area
Afghanistan       549
Yugoslav SFR      549
Sri Lanka         549
Sudan             549
Sudan (former)    549
                 ... 
Kuwait            549
Kyrgyzstan        549
Latvia            549
Lebanon           549
Zimbabwe          549
Name: count, Length: 211, dtype: int64

In [13]:
df.Element.value_counts()

Element
Yield             38613
Production        38613
Area harvested    38613
Name: count, dtype: int64

In [14]:
df.Item.value_counts()

Item
Bananas     38613
Rice        38613
Potatoes    38613
Name: count, dtype: int64

In [16]:
df.Year.value_counts()

Year
1961    1899
1992    1899
1994    1899
1995    1899
1996    1899
        ... 
1986    1899
1987    1899
1988    1899
1989    1899
2021    1899
Name: count, Length: 61, dtype: Int64

In [18]:
df.Unit.value_counts()

Unit
            46387
t           23491
ha          23418
100 g/ha    22543
Name: count, dtype: int64

In [20]:
df.Value.describe()

count    6.945200e+04
mean     1.172701e+06
std      8.939387e+06
min      0.000000e+00
25%      1.000000e+04
50%      5.330500e+04
75%      2.000000e+05
max      2.144299e+08
Name: Value, dtype: float64